In [1]:
import collections
import bisect
import glob
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib as mpl
import scipy.signal as signal

from sklearn.neighbors import KernelDensity
from sklearn.mixture import GaussianMixture

In [2]:
#plotting things

#%matplotlib qt5 -- I don't know what this is
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns

from cycler import cycler


#All of Anandh's customized seaborn/matplotlib settings

sns.set_context("talk", font_scale=1.5, rc={"lines.linewidth": 1.5})
sns.set_style("ticks")
sns.set_style({"xtick.direction": "in","ytick.direction": "in"})

#%config InlineBackend.figure_f.ormats=['svg']

mpl.rc('axes', prop_cycle=(cycler('color', ['r', 'k', 'b','g','y','m','c']) ))

mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

#mpl.rc('text', usetex=False)
#mpl.rc('text.latex', preamble=r'\usepackage{helvet}
#\renewcommand\familydefault{\sfdefault}\usepackage{sansmath}\sansmath')

    #If you want to use a different font
# mpl.rc('font',**{'family':'sans-serif','sans-serif':['Helvetica'], 
#                  'serif': ['Helvetica']})

tw = 1.5
sns.set_style({"xtick.major.size": 3, "ytick.major.size": 3,
               "xtick.minor.size": 2, "ytick.minor.size": 2,
               'axes.labelsize': 16, 'axes.titlesize': 16,
               'xtick.major.width': tw, 'xtick.minor.width': tw,
               'ytick.major.width': tw, 'ytick.minor.width': tw})

mpl.rc('xtick', labelsize=14) 
mpl.rc('ytick', labelsize=14)
mpl.rc('axes', linewidth=1.5)
mpl.rc('legend', fontsize=14)
mpl.rc('figure', figsize=(9,8))

In [3]:
def get_values(file_directory, channel='GFP/FITC-A'):
    ''' Reads in the values from a specific channel for a given flow file.
    Defaults to taking GFP/FITC-A.'''
    flow_data = pd.read_csv(file_directory)
    flow_data_gfp_values = np.log10(flow_data[channel].values)
    
    return flow_data_gfp_values

In [4]:
def make_df(file_directory, channel='GFP/FITC-A'):
    '''Creates a dataframe from the given file directory. Reads in all csvs, 
    extracts the data from the channel of interest (defaults to GFP/FITC-A), 
    and returns one dataframe.'''
    all_files = glob.glob(file_directory)
    all_files.sort()
    
    all_data = []
    for file in all_files:
        data = get_values(file, channel)
        all_data.append(data)
    
    labels = []
    for i in range(0, len(all_files)):
        #this won't get the well label right in the output df unless the file format is:
        #'../../../..\'(letter)(number).csv'
        #the forward slash is from glob filenames. this can't handle 'gated_data' at the beginning of the filename
        mini_label = str(all_files[i].split('/')[-1].split('\\')[1].split('.')[0])
        #print(mini_label)
        label = [mini_label]*len(all_data[i])
        labels.append(label)
    
    flat_all_data = [item for sublist in all_data for item in sublist]
    flat_labels = [item for sublist in labels for item in sublist]
    
    df = pd.DataFrame(dict(well=flat_labels, log10GFP=flat_all_data))
    return df

In [5]:
def get_peak_locations_from_KDE_fit(data):
    ''' Performs a KDE fit and then uses scipy.signal.find_peaks_cwt to get peaks.
        The KDE bandwith parameter is critical, and 0.25 has worked well in the past.
        If it feels like you are missing many peak calls, decrease the bandwith. If it feels
        like you are having too many peak calls, increase the bandwith. 
        
        Don't change the bandwith without good reason, it took awhile to decide on 0.25. '''
    
    kde = KernelDensity(bandwidth=0.25, kernel='gaussian')
    kde.fit(data[:, None]);

    x_range = np.linspace(0, 6, 1200)
    kde_estimates = np.exp(kde.score_samples(x_range[:, None]))

    #Use the SciPy function to get the KDE peaks
    peaks = signal.find_peaks_cwt(kde_estimates, np.arange(30, 200), min_snr=1)

    means_init = []
    
    for peak in peaks:
        means_init.append(x_range[peak])
    
    return means_init

In [6]:
def fit_GMM_KDE(data, peaks, threshold = 0.01): 
    """Generate a Gaussian mixture model from the output
    of a Gaussian Kernel Density Estimation. 
    Outputs the mean of the on peak, fraction on, mean of the off peak, 
    and fraction off. This version of the code assumes all cells not in the on peak are off!
    This is obviously only a good assumption for uni/bimodal data. If you have multimodal data,
    do not use this code."""
    
    data = data.reshape(len(data), 1)

    peaks = np.array(peaks).reshape(len(peaks), 1)
    opt_gmm = GaussianMixture(n_components = len(peaks) , means_init = peaks).fit(data)  

    labels = opt_gmm.predict(data)
    labels = np.ravel(labels.reshape(len(labels), 1))

    means = opt_gmm.means_
    
    #this df contains each measurement value and the gaussian it is associated with.
    #you can use this to pull out the measurements that fall into the desired gaussian for gating.
    df = pd.DataFrame({'fluor value': np.ravel(data), 'which_gaussian': labels})


    #df.head(10)
    counts = []
    means = []



    for i in range(0, len(peaks)):
        df_distro = df.loc[df['which_gaussian']==i]
        counts.append(len(df_distro))
        means.append(np.mean(df_distro['fluor value'].values))

    print('peaks identified in the kde fit of data: ', peaks)
    print('means of the gaussians the GMM optimized: ', means)
    print('counts in each gaussian: ', counts)
    total = len(df)
    # print('total: ', total)

    fractions = np.array(counts)/total
    print('raw fractions of data in each gaussian: ', fractions)

    ##Initializing corrected lists of means and fractions of subpopulations
    GMM_accepted_means = []

    GMM_corrected_fractions = []

    for i in range(0, len(fractions)):
        if fractions[i] > threshold: 
            GMM_accepted_means.append(means[i])
            GMM_corrected_fractions.append(fractions[i])    

    print('means of gaussians that contain fraction of data above thresh ({}): '.format(threshold), GMM_accepted_means)
    print('fraction of data in above-thresh gaussians: ', GMM_corrected_fractions)

    #uses the gaussian with the highest mean (the last one in the accepted means) as the ON gaussian
    index_of_on = GMM_accepted_means.index(max(GMM_accepted_means))

    #the threshold collects all the fractions that are above threshold, meaning its possible for the sum
    #of the collected fractions to be less than 1. This is fine because this line tosses everything except
    #the fraction of measurements in the ON gaussian and calculates the fraction in the OFF as 1-this, 
    #saying that any sub-threshold counts above this are OFF and everything below this is also OFF
    fraction_of_highest_peak = GMM_corrected_fractions[index_of_on]

    fraction_off = 1 - fraction_of_highest_peak

#     print('fraction in highest gaussian: ', fraction_of_highest_peak)
#     print('1 - fraction_in_highest_gaussian: ', fraction_off)

    mean_of_highest_peak = GMM_accepted_means[index_of_on]

    #because there can be many possible above-threshold means that are OFF (below the highest mean-ed gaussian (ON) )
    #we want to come up with an appropriate summary statistic for their value. This can be their mean mean, weighted by
    #the fraction of the data they represent
    #This means we multiply the mean of each non-ON gaussian by the fraction of measurements in that gaussian, then
    #take the average of these weighted values to get the weighted average value for all the non-ON data.
    weighted_peak_means = []
    fracs_to_use = []
    for i in range(0, len(GMM_corrected_fractions)):
        #if its the one corresponding to the highest gaussian, do nothing
        if i == index_of_on:
            pass
        #otherwise, compute the weighted average using this gaussian's data too.
        else: 
            #we retransform out of log space because avg of 2 and 3 in log is 2.5,
            #but real average is scaled by log 10 and avg value is not 10**2.5
            weighted_mean = GMM_corrected_fractions[i] * np.power(10, GMM_accepted_means[i])
            fracs_to_use.append(GMM_corrected_fractions[i])
            weighted_peak_means.append(weighted_mean)
            

    mean_of_off_population = np.log10(np.sum(weighted_peak_means)/np.sum(fracs_to_use))

    return mean_of_highest_peak, fraction_of_highest_peak, mean_of_off_population, fraction_off, df

In [7]:
def fit_GMM_KDE_wrapper (data):
    """Wrapper function to get both the peaks from a KDE fit, and then 
    from the Gaussian mixture model. Returns the mean of the broken cells, 
    and the fraction of broken cells."""
    
    peak_locations= get_peak_locations_from_KDE_fit(data)
    
    mean_of_highest_peak, fraction_of_highest_peak, \
        mean_of_off_population, fraction_off, \
        df_gaussian_distros = fit_GMM_KDE(data, peak_locations, threshold = 0.01)
    
    return mean_of_highest_peak, fraction_of_highest_peak, mean_of_off_population, fraction_off, df_gaussian_distros

In [8]:
def GMM_method(df_, wells):
    '''Wrapper function for the the entire generation of the final output df. 
    Takes the input dataframe and a list of all wells you want to perform GMM fitting on.
    '''
    means_on = []
    fractions_on = []
    means_off = []
    fractions_off = []
    wells_df = []
    dfs_gaussian_distros = []
    
    for well in wells:
        data = df_.loc[df_['well'] == well]
        
        YFP = data['log10GFP'].values
        
        mean_of_highest_peak, fraction_of_highest_peak, mean_of_off_population, \
                        fraction_off, df_gaussian_distros = fit_GMM_KDE_wrapper(YFP)
        
        means_on.append(mean_of_highest_peak)
        fractions_on.append(fraction_of_highest_peak)
        means_off.append(mean_of_off_population)
        fractions_off.append(fraction_off)
        dfs_gaussian_distros.append(df_gaussian_distros)
        wells_df.append(well) 

    plt_df = pd.DataFrame({'mean ON': means_on, 'fraction ON' : fractions_on,
                           'mean OFF':means_off, 'fraction OFF' : fractions_off,
                           'well': wells_df, 'df_gaussian_distros': dfs_gaussian_distros})
    
    return plt_df

In [9]:
# Initialize the list of wells that you have data files for and wish to fit with GMMs
first = [i+j for i in ['A', 'B', 'C'] for j in ['1','2','3', '4', '5', '6', '7', '8', '9']]
second = [i+j for i in ['D', 'E', 'F', 'G', 'H'] for j in ['1','2','3', '4', '5', '6', '7', '8']]

wells_10 = first + second


#all 96 wells
wells = [i+j for i in ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'] for j in ['1','2','3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]

In [10]:
#set some variables

syto_channel = 'mKate/APC-A'
yfp_channel = 'GFP/FITC-A'
bfp_channel = 'CFP/VioBlue-A'

In [11]:
#Read in the tidied data generated by "ADH_automatic_flow_gating_and_well_labeling.ipynb"
# The input for make_df is the directory that you want automatic fractions generated for. 
input_df = make_df('../../Local Data/20181009 top 4 A B cell vars A=B flow samples/23hr/[C][4].csv', channel=syto_channel)
# print(input_df.shape)
input_df = input_df.dropna()
# print(input_df.shape)

C:\Users\geeze\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in log10
  """


In [12]:
# Generate an output_df by running GMM method. The inputs are the input_df and
# the list of all wells you want generated
output_df = GMM_method(input_df, ['C4'])

peaks identified in the kde fit of data:  [[0.235196  ]
 [3.71809842]]
means of the gaussians the GMM optimized:  [1.7413081322762336, 3.5034089213006054]
counts in each gaussian:  [113, 9268]
raw fractions of data in each gaussian:  [0.01204562 0.98795438]
means of gaussians that contain fraction of data above thresh (0.01):  [1.7413081322762336, 3.5034089213006054]
fraction of data in above-thresh gaussians:  [0.012045624133887645, 0.9879543758661123]


C:\Users\geeze\Anaconda3\lib\site-packages\scipy\stats\stats.py:1713: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  return np.add.reduce(sorted[indexer] * weights, axis=axis) / sumval


In [13]:
# Check the output to make sure it looks correct
output_df

,mean ON,fraction ON,mean OFF,fraction OFF,well,df_gaussian_distros
0,3.503409,0.987954,1.741308,0.012046,C4,fluor value which_gaussian 0 1.7...


In [93]:
# Write the output file to a csv. 
output_df.to_csv('../../Local Data/20181009 top 4 A B cell vars A=B flow samples/23hr/C4_TEST.csv', index = False)